In [1]:
import sys, os
sys.path.append('../MultiFishLSS/')
from headers import *
from twoPoint import *
from twoPointNoise import *
from classy import Class


## Generate $M_\nu$ forecasts with full-shape power spectrum and CMB lensing

#### Fiducial cosmology

In [2]:
default_cosmo = {
        'output': 'tCl lCl mPk',
        'non linear':'halofit',
        'l_max_scalars': 4000,
        'lensing': 'yes',
        'P_k_max_h/Mpc': 2.,
        'z_pk': '0.0,1087',
        'A_s': 2.10732e-9,
        'n_s': 0.96824,
        'alpha_s': 0.,
        'h': 0.6736,
        'N_ur': 2.0328,
        'N_ncdm': 1,
        'm_ncdm': 0.06,
        'tau_reio': 0.0544,
        'omega_b': 0.02237,
        'omega_cdm': 0.1200,
        'Omega_k': 0.,
        'Omega_Lambda': 0.,
        'w0_fld':-1,
        'wa_fld':0}
cosmo = Class()
cosmo.set(default_cosmo)
cosmo.compute()
h = cosmo.h()

chi = lambda zz: cosmo.comoving_distance(zz)*h

#### Experiment configuration

In [3]:
fsky = 5000/41253 # 5000 deg2

# two redshift bins from z = 2.7 to 3.3
zmin, zmax = 2.7, 3.3
nbins = 2

# 2 galaxy samples 
# name, bias, number density as lists
samples=['ga','gb']
b = [lambda z: 2.5, lambda z: 3.5] 
n = [lambda z: 2e-4, lambda z: 2e-4]

# overlap between stochastic terms
# index 0 and 2 are auto-correlations, so they must be 1
# index 1 is the cross-correlation between the two samples, set to 0 by default for no overlap
fover=[1,0,1]

# experiment name and basedir
bd='./'
surveyname='example'

exp = experiment(zmin=zmin, zmax=zmax, nbins=nbins, fsky=fsky, b=b, n=n,samples=samples,fover=fover)
forecast = fisherForecast(experiment=exp,cosmo=cosmo,name=surveyname,basedir=bd,ell=np.arange(10,3000,1))    

### Full-shape

#### Compute derivatives (if not done already)

Derivatives are computed faster with dedicated node, using `example_setup.sh`


In [4]:
basis    = ['m_ncdm','h','log(A_s)','n_s','omega_cdm','omega_b','tau_reio']
n_global = len(basis)
basis   += ['b','N','alpha0','b2','bs','N2','N4','alpha2','alpha4']
basis    = np.array(basis)

forecast.free_params = basis
forecast.compute_derivatives() # will not overwrite unless specified

#### Compute Fisher matrix

In [5]:
kmax_knl    = .5 # ratio of kmax to k_nl(z)
derivatives = forecast.load_derivatives(basis)
F = forecast.gen_fisher(basis,n_global,kmax_knl=kmax_knl,derivatives=derivatives)

#### Add CMB primary and BBN prior 

In [6]:
# Planck_SO prior on cosmology
inputdir = os.path.join('..','MultiFishLSS','input')
cmb_fn = os.path.join(inputdir,'Planck_SO.txt')
cmb_prior = np.loadtxt(cmb_fn)
cmb_basis = np.array(['h','log(A_s)','n_s','omega_cdm','omega_b','tau_reio','m_ncdm','N_ur','alpha_s','Omega_k'])
idxs = []
for i,param in enumerate(basis[:n_global]):
    idx = np.where(cmb_basis==param)[0][0]
    idxs.append(idx)
F[:n_global][:,:n_global] += cmb_prior[idxs][:,idxs]

# conservative BBN prior on omega_b
omega_b_idx = np.where(basis=='omega_b')[0][0]
F[omega_b_idx,omega_b_idx] += 1/(0.0005**2)

#### Compute neutrino mass constraints

In [7]:
Finv    = np.linalg.inv(F)
Mnu_err = np.sqrt(np.diag(Finv))[0]
print(f"Neutrino mass constraint: {Mnu_err*1e3:.3f} meV")

Neutrino mass constraint: 78.498 meV


### Full-shape + CMB lensing

#### Compute derivatives (if not done already)

In [8]:
basis = np.array(['m_ncdm','h','log(A_s)','n_s','omega_cdm','omega_b','tau_reio',
        'b','N','alpha0','b2','bs','N2','N4','alpha2','alpha4'])
basis_Cl = np.array(['m_ncdm','h','log(A_s)','n_s','omega_cdm','omega_b','tau_reio',
        'b','N','alpha0','b2','bs','alpha0k'])
# number of global parameters
n_global = 7

# number of shared parameters between P(k,mu) and C_ell
# assuming 2 tracers:
# 2 b1's+3 N's+2 alpha0's+2 b2's+2 bs's = 11 shared params/bin
n_shared = n_global + 11*forecast.experiment.nbins

In [9]:
# compute P(k,mu) derivatives
forecast.free_params = basis
forecast.compute_derivatives()

# compute C_ell derivatives
forecast.free_params = basis_Cl
forecast.compute_Cl_derivatives()

#### Compute Fisher matrices

In [10]:
# Fisher matrix from P(k,mu)
kmax_knl = .5 # ratio of kmax to k_nl(z)
derivatives   = forecast.load_derivatives(basis)
F = forecast.gen_fisher(basis,n_global,kmax_knl=kmax_knl,derivatives=derivatives,kpar_min=1e-3)

# Fisher matrix from C_ell
F_Cl=forecast.gen_lensing_fisher(basis_Cl,n_global,ell_min=30,ell_max=None,CMB='SO')


#### Combine Fisher matrices

In [11]:
fishers    = [F,F_Cl]
F_combined = forecast.combine_fishers(fishers,n_shared)


#### Add CMB lensing and BBN prior

In [12]:
# Planck_SO prior on cosmology
inputdir = os.path.join('..','MultiFishLSS','input')
cmb_fn = os.path.join(inputdir,'Planck_SO.txt')
cmb_prior = np.loadtxt(cmb_fn)
cmb_basis = np.array(['h','log(A_s)','n_s','omega_cdm','omega_b','tau_reio','m_ncdm','N_ur','alpha_s','Omega_k'])
idxs = []
for i,param in enumerate(basis[:n_global]):
    idx = np.where(cmb_basis==param)[0][0]
    idxs.append(idx)
F_combined[:n_global][:,:n_global] += cmb_prior[idxs][:,idxs]

# conservative BBN prior on omega_b
omega_b_idx = np.where(basis=='omega_b')[0][0]
F_combined[omega_b_idx,omega_b_idx] += 1/(0.0005**2)

#### Compute neutrino mass constraints

In [13]:
Finv    = np.linalg.inv(F_combined)
Mnu_err = np.sqrt(np.diag(Finv))[0]
print(f"Neutrino mass constraint: {Mnu_err*1e3:.3f} meV")

Neutrino mass constraint: 34.690 meV
